In [1]:
import requests
import datetime
from uuid import uuid4
from flask import request, jsonify, Flask
import json
import hashlib
from urllib.parse import urlparse

In [9]:
class TestCoin:
    
    def __init__(self):
        self.chain = []
        self.transactions = []
        self.addBlock(proof = 1, previousHash = "0")
        self.nodes = set()
    
    def addBlock(self, proof, prevHash):
        block = {
            "index": len(self.chain) + 1,
            "timestamp": str(datetime.datetime.now()),
            "proof": proof,
            "prevhash": prevHash
        }
        self.transactions = []
        self.chain.append(block)
        return block
    
    def getLastBlock(self):
        return self.chain[-1]
    
    def addNode(self, address):
        addressIP = urlparse(address)
        self.nodes.add(addressIP.path)
    
    def proofOFWork(self, prevProof):
        newProof = 1
        solved = False
        
        while solved is False:
            hashProblem = hashlib.sha256(str(newProof ** 2 - prevProof ** 2).encode()).hexdigest()
            if hashProblem[:4] == "0000":
                solved = True
            else:
                newProof += 1
        return newProof
    
    def hashBlock(self, block):
        encodedBlock = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha3_256(encodedBlock).hexdigest()
    
    def addTransction(self, sender, receiver, amount):
        self.transacitons.append({
            "sender": sender,
            "receiver": receiver,
            "amount": amount
        })
        previousBlock = self.getLastBlock()
        return previousBlock["index"] + 1
    
    def isChainValid(self, chain):
        prevuousBlock = chain[0]
        current_index = 1
        
        while current_index < len(chain):
            currentBlock = chain[current_index]
            currentPreviousHash = currentBlock["prevHash"]
            if currentPreviousHash != self.hashBlock(prevuousBlock):
                return False
            currentProof = currentBlock["proof"]
            previousProod = prevuousBlock["proof"]
            hashJob = hashlib.sha256(str(currentProof ** 2 - previousProod ** 2).encode()).hexdigest()
            if hashJob[:4] != "0000":
                return False
            previousBlock = currentBlock
            current_index += 1
        return True
        
    def shouldReplaceChain(self):
        network = self.nodes
        currentLength = len(self.chain)
        longestChain = None
        
        for node in network:
            nodeResponse = requests.get(f"http://{node}/chain")
            if nodeResponse.status_code == 200:
                chain = nodeResponse.json()["chain"]
                length = nodeResponse.json()["length"]
                if length > currentLength and self.isChainValid(chain):
                    longestChain = chain
                    currentLength = length
            if longestChain:
                self.chain = chain 
                return True
        return False